In [146]:
df_raw = spark.read.options(header='False',wholetext=True)\
    .text('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp.xml')
#df_raw['value_1']=df_raw['value'].apply(lambda x: ' '.join(x) )

#df_text=' '.join(df_raw['value_1'])
#df_raw.write.mode("overwrite").saveAsTable("default.dblp")
df_raw.show()

StatementMeta(sparkpool3, 13, 3, Finished, Available)

+--------------------+
|               value|
+--------------------+
|<?xml version="1....|
|<!DOCTYPE dblp SY...|
|              <dblp>|
|<phdthesis mdate=...|
|<author>Carmen He...|
|<title>Modell zur...|
|   <year>2010</year>|
|<school>Aarhus Un...|
|<pages>1-315</pages>|
|<isbn>978-3-86596...|
|<ee>http://d-nb.i...|
|</phdthesis><phdt...|
|<author>Gerd Hoff...|
|<title>Ein Verfah...|
|   <year>2002</year>|
|<school>Universit...|
|<ee>http://ubt.op...|
|<ee>https://nbn-r...|
|<ee>http://d-nb.i...|
|        </phdthesis>|
+--------------------+
only showing top 20 rows

In [147]:
df=df_raw.select('value').collect()

StatementMeta(sparkpool3, 13, 4, Finished, Available)

In [148]:
mstr=""
flag=False
dblp=[]
for row in df:
    s=str(row.value)
    if s.__contains__('</phdthesis>'):
        i=s.index('</phdthesis>')
        s1=s[0:i+len('</phdthesis>')]
        mstr=mstr+s1
        dblp.append(mstr)
        mstr=""
        flag=False
    if s.__contains__('<phdthesis'):
        i=s.index('<phdthesis')
        mstr=mstr+s[i:]
        flag=True
    else:
        if flag==True:
            mstr=mstr+s

StatementMeta(sparkpool3, 13, 5, Finished, Available)

In [149]:
dblp[:2]

StatementMeta(sparkpool3, 13, 6, Finished, Available)

['<phdthesis mdate="2016-05-04" key="phd/dk/Heine2010"><author>Carmen Heine</author><title>Modell zur Produktion von Online-Hilfen.</title><year>2010</year><school>Aarhus University</school><pages>1-315</pages><isbn>978-3-86596-263-8</isbn><ee>http://d-nb.info/996064095</ee></phdthesis>', '<phdthesis mdate="2020-02-12" key="phd/Hoff2002"><author>Gerd Hoff</author><title>Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch</title><year>2002</year><school>University of Trier, Germany</school><ee>http://ubt.opus.hbz-nrw.de/volltexte/2004/146/</ee><ee>https://nbn-resolving.org/urn:nbn:de:hbz:385-1468</ee><ee>http://d-nb.info/971713243</ee></phdthesis>']

In [150]:
import xml.etree.ElementTree as ET 
list_dblp=[]
for d in dblp:
    dict_dblp={}
    try:
        tree = ET.fromstring(d.replace('&','')) 
        for child in tree:
            #print(child.tag, ' '.join(child.itertext()))
            dict_dblp[child.tag]=' '.join(child.itertext())
        list_dblp.append(dict_dblp)
    except :
        print(d)

list_dblp[:2]

StatementMeta(sparkpool3, 13, 7, Finished, Available)

[{'author': 'Carmen Heine', 'title': 'Modell zur Produktion von Online-Hilfen.', 'year': '2010', 'school': 'Aarhus University', 'pages': '1-315', 'isbn': '978-3-86596-263-8', 'ee': 'http://d-nb.info/996064095'}, {'author': 'Gerd Hoff', 'title': 'Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch', 'year': '2002', 'school': 'University of Trier, Germany', 'ee': 'http://d-nb.info/971713243'}]

In [168]:
import xml.etree.ElementTree as ET 
list_dblp_together=[]
for d in dblp:
    dict_dblp={}
    try:
        tree = ET.fromstring(d.replace('&','')) 
        #dict_dblp[child.tag]=' '.join(child.itertext())
        list_dblp_together.append([' '.join(tree.itertext())])
    except :
        print(d)

list_dblp_together[:2]

StatementMeta(sparkpool3, 13, 25, Finished, Available)

[['Carmen Heine Modell zur Produktion von Online-Hilfen. 2010 Aarhus University 1-315 978-3-86596-263-8 http://d-nb.info/996064095'], ['Gerd Hoff Ein Verfahren zur thematisch spezialisierten Suche im Web und seine Realisierung im Prototypen HomePageSearch 2002 University of Trier, Germany http://ubt.opus.hbz-nrw.de/volltexte/2004/146/ https://nbn-resolving.org/urn:nbn:de:hbz:385-1468 http://d-nb.info/971713243']]

In [178]:
columns = ["value"]

df_all = spark.createDataFrame(data=list_dblp_together,schema =columns)
df_all.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_all.txt')

df_sample = spark.createDataFrame(data=list_dblp_together[:500],schema =columns)
df_sample.coalesce(1).write.format("text").option("header", "false").mode("overwrite").save('adl://cp-bizops-c15.azuredatalakestore.net/local/users/lowen/UIUC/dblp_sample.txt')

print(len(list_dblp),len(list_dblp_together),df_all.count(),df_sample.count())

StatementMeta(sparkpool3, 13, 35, Finished, Available)

78704 78704 78704 500

In [171]:
display(df_all.limit(5))

StatementMeta(sparkpool3, 13, 28, Finished, Available)

SynapseWidget(Synapse.DataFrame, df13e881-31b7-4f4e-9e46-04d2a61e321e)

In [143]:
#import pandas as pd 
#df_dblp=pd.DataFrame.from_dict(list_dblp[:1])

#i=0
#for row in list_dblp[1:]:
#    df_dblp=df_dblp.append(row,ignore_index=True)
#    i=i+1
#    if i>2:
#        break

StatementMeta(sparkpool3, 12, 58, Finished, Cancelled)

In [163]:
#import pandas as pd 
#df_dblp=pd.DataFrame.from_dict(list_dblp[:1])
#display(df_dblp)

StatementMeta(sparkpool3, 13, 20, Finished, Available)

SynapseWidget(Synapse.DataFrame, a8f2686c-e46d-48fa-81cb-3acc3864e767)


/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)